In [1]:
# libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
    
import re

from tqdm import tqdm
import time

import sqlite3

import os
import random
import string

In [2]:
# PANDAS OPTIONS
# Set maximum number of columns and rows to display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Set the maximum column width to a high value
pd.set_option('display.max_colwidth', 1000)

In [3]:
# read back raw data from csv
df = pd.read_csv('scraped_data_clean.csv')

#df.head(3)

In [ ]:
df.head(2)

In [4]:
# Function to extract numbers divisible by 50 from a given text
def extract_divisible_by_50(text):
    numbers = re.findall(r'\b(?:[5-9][0]|50)\b', text)
    return [int(num) for num in numbers]

# Apply the function to "product_name" and "product_description" columns and store the result in "ammo_qtty" column
df['ammo_qtty'] = df['product_name'].apply(extract_divisible_by_50) + df['product_description'].apply(extract_divisible_by_50)

# Filter and keep only the numbers divisible by 50 without remainders
df['ammo_qtty'] = df['ammo_qtty'].apply(lambda x: [num for num in x if num % 50 == 0])

                                                                                              product_name  \
0                                  BOITE DE 325 CARTOUCHES 22LR FEDERAL GOLD PLATED HOLLOW POINT 36 GRAINS   
1                                                                 Boîte ancienne 22 LR Remington 22 Target   
2                           boite de 325 cartouches de 22Lr de marque Federal , Automatch , pour semi auto   
3                                                                   Boîte ancienne 22 LR Fiocchi expansive   
4                                                                        Boîte ancienne 22 LR NCS 20 Match   
5                                                                              Boîte 22 LR ancienne NCS 30   
6                                                                  Boîte ancienne 22 LR Fiocchi ultrasonic   
7                                                                                       Boite ancienne T22   
8         

In [8]:
df.ammo_qtty[:10]

0          []
1          []
2          []
3          []
4          []
5          []
6          []
7          []
8        [50]
9    [50, 50]
Name: ammo_qtty, dtype: object

In [ ]:
df.columns

### old solution 1

In [ ]:
## regex to catch any number divisible by 50 (min qtty of rounds in a box of ammo)
#def extract_bullet_qtty(text):
#    # match any number that is divisible by 50 without remainder
#    regex = r"\b(0|[5-9]\d*[0]|100)\s*(?:boites de\s*)?(?:cartouches|balles|munitions)\b"
#    match = re.search(regex, text, re.IGNORECASE)
#    if match:
#        # extract the matched number and convert it to integer
#        qtty = int(match.group(1))
#        # round the quantity to the nearest 50
#        qtty = (qtty // 50) * 50
#        return qtty
#    else:
#        return None
#
## check titles with regex
#df['bullet_qtty'] = df['product_name'].apply(extract_bullet_qtty)
#
## check for empty cells, if any do a pass of regex on product description --- !!! DUPE avoidance !!!
#df.loc[df['bullet_qtty'].isnull(), 'bullet_qtty'] = df['product_description'].apply(extract_bullet_qtty)
#
## if still no data, we fill with 50 for default min number of ammo per box
#df['bullet_qtty'].fillna(50, inplace=True)

In [ ]:
# qtty of ammo search in TITLE

# Regular expression pattern for finding numbers divisible by 50 without remainder
pattern1 = r'\b(?:[5-9]|[1-9]\d{1,})0\b|\b525\b'

# Function to find matching numbers and store in new columns
def find_ammo_qtty_title(df):
    # Find all matching numbers and store them in a list
    matches = df['product_name'].apply(lambda x: re.findall(pattern1, str(x)))
    
    # Create new columns for each match and store the results
    for i, match in enumerate(matches):
        for j in range(len(match)):
            column_name = f'ammo_qtty_title_{j+1}'
            df.at[i, column_name] = match[j]
    
    return df

In [ ]:
find_ammo_qtty_title(df)

In [ ]:
df_new.head(3)

In [ ]:
# qtty of ammo search in DESCRIPTION

# Regular expression pattern for finding numbers divisible by 50 without remainder
pattern2 = r'\b(?:[5-9]|[1-9]\d{1,})0\b|\b525\b'

# Function to find matching numbers and store in new columns
def find_ammo_qtty_description(df):
    # Find all matching numbers and store them in a list
    matches = df['product_description'].apply(lambda x: re.findall(pattern2, str(x)))
    
    # Create new columns for each match and store the results
    for i, match in enumerate(matches):
        for j in range(len(match)):
            column_name = f'ammo_qtty_description_{j+1}'
            df.at[i, column_name] = match[j]
    
    return df

In [ ]:
df_new2 = find_ammo_qtty_description(df)
df_new2.head(3)

In [ ]:
# qtty of boxes search

# Regular expression pattern for finding numbers before/after French words that mean "box" or "boxes"
pattern3 = r'\b(\d+)\s*(?:(?:LOT|PAQUET)\s+DE\s+)?(?:(\d+)\s*(?:BOITE|BOITES))\b|\b(?:PAR\s+(\d+))\b'

# Function to find matching numbers and store in new columns
def find_box_numbers(df):
    # Find all matching numbers and store them in a list
    matches_name = df['product_name'].apply(lambda x: re.findall(pattern3, str(x)))
    matches_desc = df['product_description'].apply(lambda x: re.findall(pattern3, str(x)))
    matches = matches_name + matches_desc
    
    # Create new columns for each match and store the results
    for i, match in enumerate(matches):
        for j in range(len(match)):
            column_name = f'box_qty_{j+1}'
            df.at[i, column_name] = ''.join(match[j])
    
    return df

In [ ]:
df_new3 = find_box_numbers(df)
df_new3.head(3)

In [ ]:
df.head()

In [ ]:
# V2 for bullet quantities

# col1 for qtty from title and description as a flat number
# col2 for num of boxes * 50
# if both are None we set default number of 50 in col3

def extract_bullet_qtty(text):
    # Check 1: total amount of bullets
    regex1 = r"\b(\d+)\s*boîte(?:s)?\s*de\s*(50|525)\s*(?:cartouches|balles|munitions)\b|\bMunition \/ boite\s*:\s*(\d+)\s*x\s*(50|525)\b|\bQuantité\s*:\s*(50|525)\b"
    match1 = re.search(regex1, text, re.IGNORECASE)
    if match1:
        qtty = int(match1.group(1)) if match1.group(1) is not None else int(match1.group(2))
        return qtty
    # Check 2: amount of boxes
        regex2 = r"\bLOT DE (\d+) BOITES\b"
    match2 = re.search(regex2, text, re.IGNORECASE)
    if match2:
        qtty = int(match2.group(1)) * 50
        return qtty
    else:
        return 0

# create new columns for ammo quantity
df["bullet_qtty_v1"] = df.apply(lambda x: extract_bullet_qtty(x["product_name"]) or extract_bullet_qtty(x["product_description"]) or 0)
df["bullet_qtty_v2"] = df.apply(lambda x: extract_bullet_qtty(x["product_name"]) * 50 if re.search(r"\b(\d+)\s*boites\b", x["product_name"]) else extract_bullet_qtty(x["product_description"]) * 50 if re.search(r"\b(\d+)\s*boites\b", x["product_description"]) else 0, axis=1)

# set default value of 50 for products with missing values
df["bullet_qtty_default"] = df.apply(lambda x: 50 if x["bullet_qtty_v1"] == 0 and x["bullet_qtty_v2"] == 0 else None, axis=1)

df.head(5)